In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, BertModel
import torch
import joblib

# Cargar el dataset
df = pd.read_csv('youtoxic_english_1000.csv')

# Preparar los datos
X = df['Text']
y = df['IsToxic']

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializar el modelo y tokenizer de BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Función para vectorizar texto usando BERT
def bert_vectorize(texts):
    encoded_input = tokenizer(list(texts), padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        model_output = model(**encoded_input)
    # Usar los embeddings de la capa [CLS] como representación del texto
    return model_output.last_hidden_state[:, 0, :].numpy()

# Vectorizar el texto
X_train_vectors = bert_vectorize(X_train)
X_test_vectors = bert_vectorize(X_test)

# Entrenar un modelo (Random Forest en este caso)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_vectors, y_train)

# Evaluar el modelo en el conjunto de entrenamiento
y_train_pred = clf.predict(X_train_vectors)
train_accuracy = accuracy_score(y_train, y_train_pred)
print("Precisión en el conjunto de entrenamiento:", train_accuracy)

# Evaluar el modelo en el conjunto de prueba
y_test_pred = clf.predict(X_test_vectors)
test_accuracy = accuracy_score(y_test, y_test_pred)
print("Precisión en el conjunto de prueba:", test_accuracy)

# Calcular el overfitting exacto
overfitting = train_accuracy - test_accuracy
print("Overfitting exacto: ", overfitting)

# Imprimir el informe de clasificación
print("\nInforme de clasificación en el conjunto de prueba:")
print(classification_report(y_test, y_test_pred))

# Guardar el modelo en un archivo utilizando joblib
joblib_file = "esencial_model.joblib"
joblib.dump(clf, joblib_file)
print(f"Modelo guardado en {joblib_file}")


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Precisión en el conjunto de entrenamiento: 1.0
Precisión en el conjunto de prueba: 0.71
Overfitting exacto:  0.29000000000000004

Informe de clasificación en el conjunto de prueba:
              precision    recall  f1-score   support

       False       0.64      0.88      0.74        93
        True       0.85      0.56      0.67       107

    accuracy                           0.71       200
   macro avg       0.74      0.72      0.71       200
weighted avg       0.75      0.71      0.70       200

Modelo guardado en esencial_model.joblib
